In [42]:
import os
import pandas as pd
import numpy as np

PROCESSED_DIR = "./data/processed"
REPORT_DIR = "./reports"
os.makedirs(REPORT_DIR, exist_ok=True)

In [44]:
orders = pd.read_parquet(PROCESSED_DIR + "/orders.parquet")
customers = pd.read_parquet(PROCESSED_DIR + "/customers.parquet")
items = pd.read_parquet(PROCESSED_DIR + "/order_items.parquet")
payments = pd.read_parquet(PROCESSED_DIR + "/order_payments.parquet")
reviews = pd.read_parquet(PROCESSED_DIR + "/order_reviews.parquet")
products = pd.read_parquet(PROCESSED_DIR + "/products.parquet")
sellers = pd.read_parquet(PROCESSED_DIR + "/sellers.parquet")
geo = pd.read_parquet(PROCESSED_DIR + "/geolocation.parquet")
catmap = pd.read_parquet(PROCESSED_DIR + "/category_translation.parquet")

print("orders:", orders.shape)
print("customers:", customers.shape)
print("items :", items.shape)
print("payments :", payments.shape)
print("reviews:", reviews.shape)
print("products :", products.shape)
print("sellers :", sellers.shape)
print("geo :", geo.shape)
print("catmap:", catmap.shape)

orders: (99441, 11)
customers: (99441, 5)
items : (112650, 7)
payments : (103886, 5)
reviews: (99224, 7)
products : (32951, 9)
sellers : (3095, 4)
geo : (1000163, 5)
catmap: (71, 2)


In [46]:
# Check of missing values in important datetime columns of orders
def null_report(df, name, topn=10):
    rep = df.isnull().mean().sort_values(ascending=False).head(topn).to_frame("null_rate")
    rep.to_csv(f"{REPORT_DIR}/nulls_{name}.csv")
    print(f"Saved {REPORT_DIR}/nulls_{name}.csv")
    return rep

null_report(orders, "orders")
null_report(items, "items")
null_report(payments, "payments")
null_report(reviews, "reviews")
null_report(products, "products")

Saved ./reports/nulls_orders.csv
Saved ./reports/nulls_items.csv
Saved ./reports/nulls_payments.csv
Saved ./reports/nulls_reviews.csv
Saved ./reports/nulls_products.csv


,null_rate
product_category_name,0.018512
product_name_lenght,0.018512
product_description_lenght,0.018512
product_photos_qty,0.018512
product_weight_g,0.000061
product_length_cm,0.000061
product_height_cm,0.000061
product_width_cm,0.000061
product_id,0.000000


In [48]:
# Delivery duration in days (purchase → customer received)
if "actual_delivery_days" not in orders.columns:
    orders["actual_delivery_days"] = (orders["order_delivered_customer_date"] - orders["order_purchase_timestamp"]).dt.days
    
# Delay compared to estimated delivery date (positive = late)
if "delivery_delay_days" not in orders.columns:
    orders["delivery_delay_days"] = (orders["order_delivered_customer_date"] - orders["order_estimated_delivery_date"]).dt.days

# Boolean flag: was the order late?
if "is_late" not in orders.columns:
    orders["is_late"] = orders["delivery_delay_days"] > 0

# Show summary stats for these new variables
display(orders[["actual_delivery_days","delivery_delay_days","is_late"]].describe(include="all"))
print("Late rate:", round(orders["is_late"].mean(), 4))
orders["order_status"].value_counts().to_frame("count").head(10)

,actual_delivery_days,delivery_delay_days,is_late
count,96476.000000,96476.000000,99441
unique,NaN,NaN,2
top,NaN,NaN,False
freq,NaN,NaN,92906
mean,12.094086,-11.876881,NaN
std,9.551746,10.183854,NaN
min,0.000000,-147.000000,NaN
25%,6.000000,-17.000000,NaN
50%,10.000000,-12.000000,NaN
75%,15.000000,-7.000000,NaN


Late rate: 0.0657


,count
order_status,
delivered,96478
shipped,1107
canceled,625
unavailable,609
invoiced,314
processing,301
created,5
approved,2


In [50]:
# Aggregate items per order: count items, count sellers, sum of price & freight
order_item_agg = items.groupby("order_id").agg(
    n_items=("order_item_id","count"),
    n_sellers=("seller_id","nunique"),
    n_products=("product_id","nunique"),
    price_sum=("price","sum"),
    freight_sum=("freight_value","sum")
).reset_index()

# Aggregate payments per order
order_pay_agg = payments.groupby("order_id").agg(
    pay_total=("payment_value","sum"),
    pay_types=("payment_type","nunique"),
    installments_max=("payment_installments","max")
).reset_index()

# Merge into orders
orders_eda = orders.merge(order_item_agg, on="order_id", how="left") \
                   .merge(order_pay_agg, on="order_id", how="left")

orders_eda[["n_items","n_sellers","n_products","price_sum","freight_sum","pay_total","installments_max"]].describe()


,n_items,n_sellers,n_products,price_sum,freight_sum,pay_total,installments_max
count,98666.000000,98666.000000,98666.000000,98666.000000,98666.000000,99440.000000,99440.000000
mean,1.141731,1.013622,1.038098,137.754076,22.823562,160.990267,2.930521
std,0.538452,0.122297,0.226456,210.645145,21.650909,221.951257,2.715685
min,1.000000,1.000000,1.000000,0.850000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,1.000000,45.900000,13.850000,62.010000,1.000000
50%,1.000000,1.000000,1.000000,86.900000,17.170000,105.290000,2.000000
75%,1.000000,1.000000,1.000000,149.900000,24.040000,176.970000,4.000000
max,21.000000,5.000000,8.000000,13440.000000,1794.960000,13664.080000,24.000000


In [52]:
# Extract time features for exploratory analysis
orders_eda["year"]  = orders_eda["order_purchase_timestamp"].dt.year
orders_eda["month"] = orders_eda["order_purchase_timestamp"].dt.month
orders_eda["dow"]   = orders_eda["order_purchase_timestamp"].dt.dayofweek
orders_eda["hour"]  = orders_eda["order_purchase_timestamp"].dt.hour

# Save distributions to CSV for later visualization
orders_eda["year"].value_counts().sort_index().to_csv(f"{REPORT_DIR}/dist_year.csv")
orders_eda["month"].value_counts().sort_index().to_csv(f"{REPORT_DIR}/dist_month.csv")
orders_eda["dow"].value_counts().sort_index().to_csv(f"{REPORT_DIR}/dist_dow.csv")
orders_eda["hour"].value_counts().sort_index().to_csv(f"{REPORT_DIR}/dist_hour.csv")

orders_eda.groupby(["year","month"]).size().to_frame("orders").to_csv(f"{REPORT_DIR}/orders_by_year_month.csv")
print("Saved time distribution CSVs to", REPORT_DIR)


Saved time distribution CSVs to ./reports


In [54]:
# Check duplicates
dup_orders = orders["order_id"].duplicated().sum()
dup_items  = items[["order_id","order_item_id"]].duplicated().sum()
print("Duplicate order_id:", dup_orders, " | Duplicate (order_id, order_item_id):", dup_items)

# Check for negative values
print("Any negative price_sum?", (orders_eda["price_sum"] < 0).sum())
print("Any negative freight_sum?", (orders_eda["freight_sum"] < 0).sum())
print("Any negative actual_delivery_days?", (orders_eda["actual_delivery_days"] < 0).sum())

# Show some extreme outliers (delivery > 90 days)
orders_eda.loc[orders_eda["actual_delivery_days"] > 90, 
               ["order_id","order_purchase_timestamp","order_delivered_customer_date","actual_delivery_days"]
              ].head(10)


Duplicate order_id: 0  | Duplicate (order_id, order_item_id): 0
Any negative price_sum? 0
Any negative freight_sum? 0
Any negative actual_delivery_days? 0


,order_id,order_purchase_timestamp,order_delivered_customer_date,actual_delivery_days
1621,a4efaffc506a395c9cea7402b078c1e5,2018-01-17 10:50:09,2018-05-07 19:25:33,110.0
3077,8b7fd198ad184563c231653673e75a7f,2017-11-14 10:04:27,2018-02-27 18:05:08,105.0
3202,4f39a94d6e474819d898d6df7d394996,2017-04-28 16:28:03,2017-09-19 13:54:18,143.0
4666,b31c7dea63bb08f8cdd1ec32514ccf0b,2017-09-26 18:35:35,2018-02-05 21:25:43,132.0
7740,00d1289d5125017a90e528b5a7cee91f,2018-03-15 11:11:52,2018-06-19 18:42:26,96.0
10383,3602a80b09d914236f74c733631f3b8b,2018-01-31 20:38:38,2018-05-18 11:06:52,106.0
11399,47b40429ed8cce3aee9199792275433f,2018-01-03 09:44:01,2018-07-13 20:51:31,191.0
12813,c2a550cc5f966506b717532441c221e5,2018-01-12 15:38:34,2018-05-23 20:56:25,131.0
13672,d8dbb44d7c5b1fd8e7f41b49e27053d7,2018-05-07 20:03:32,2018-08-20 17:14:55,104.0
13815,77f1ce3b8d86fff49b9c811f00f4dc13,2018-02-22 12:33:43,2018-05-30 23:50:44,97.0


In [56]:
# Late delivery rate
late_rate = pd.DataFrame({"metric":["late_rate"], "value":[orders["is_late"].mean()]})

# Order status distribution
status_dist = orders["order_status"].value_counts().to_frame("count").reset_index().rename(columns={"index":"order_status"})

# Numerical summaries
num_summary = orders_eda[["actual_delivery_days","delivery_delay_days","n_items","price_sum","freight_sum","pay_total"]].describe().T

# Save to CSV
late_rate.to_csv(f"{REPORT_DIR}/kpi_late_rate.csv", index=False)
status_dist.to_csv(f"{REPORT_DIR}/dist_order_status.csv", index=False)
num_summary.to_csv(f"{REPORT_DIR}/num_summary.csv")

print("Saved EDA summaries to", REPORT_DIR)


Saved EDA summaries to ./reports
